In [ ]:
import seaborn as sns
import pandas as pd
import os
import shutil
tmp_dir = "./tmp_csv/"

Buscamos aerolineas con mucha diferencia entre delay de arribos y cancelaciones en un año en particular

In [ ]:
def carrier_cancelled(row):
    canc = row["Cancelled"] and row["CancellationCode"] == 'A'
    return int(canc)

def get_grouped(yrs):    
    df_acc = pd.DataFrame(columns=["UniqueCarrier", "Year", "ArrDelay", "CarrierCancelled"]).set_index(["UniqueCarrier", "Year"])
    for year in yrs:
        print("Leyendo csv: {}".format(year))
        df = pd.read_csv("../data/"+year+".csv.bz2", compression="bz2", \
                         usecols=["UniqueCarrier", "Year", "ArrDelay", "Cancelled", "CancellationCode"], \
                         encoding="ISO 8859-1")    
        
        df['CarrierCancelled'] = df.apply (lambda row: carrier_cancelled(row), axis=1)
        df = df.drop(columns=["Cancelled", "CancellationCode"])        
        df = df.groupby(by=["UniqueCarrier", "Year"]).mean()
                
        df_acc = pd.concat([df_acc, df])
        del df 
    print(df_acc)
    return df_acc

years = [str(year) for year in range(2004,2009)]

if not os.path.exists(tmp_dir+"carriers/"):
    os.makedirs(tmp_dir+"carriers/")
    
if not os.path.exists(tmp_dir+"carriers/dls_and_cxls.csv"):
    df = get_grouped(years)
    df.to_csv(tmp_dir+"carriers/dls_and_cxls.csv")

In [ ]:
df = pd.read_csv(tmp_dir+"carriers/dls_and_cxls.csv")

In [ ]:
df['Delay_Rank'] = df.reset_index().groupby('Year')['ArrDelay'].rank(ascending=False)
df['Cancel_Rank'] = df.reset_index().groupby('Year')['CarrierCancelled'].rank(ascending=False)
df[df.Year==2004]

## Comparar offsets

In [ ]:
def rank_offset(row):
    return abs(row["Delay_Rank"] - row['Cancel_Rank'])

df['Rank_Offset'] = df.apply (lambda row: rank_offset(row), axis=1)
df[df.Year == 2004]

In [ ]:
df_carriers = pd.read_csv("../data/carriers.csv")
df = df.merge(df_carriers, left_on='UniqueCarrier', right_on='Code')
print(df)

In [ ]:
df.nlargest(5, 'Rank_Offset')

In [ ]:
def get_count(yrs):    
    df_acc = pd.DataFrame(columns=["UniqueCarrier", "FlightNum"])
    for year in yrs:
        print("Leyendo csv: {}".format(year))
        df = pd.read_csv("../data/"+year+".csv.bz2", compression="bz2", \
                         usecols=["UniqueCarrier", "FlightNum"], \
                         encoding="ISO 8859-1")  
        
        df = df.groupby(by=["UniqueCarrier"]).count().reset_index()
        df_acc = pd.concat([df_acc, df], sort=True).groupby(by=["UniqueCarrier"]).count().reset_index()

    return df_acc

years = [str(year) for year in range(2004,2009)]

df_count = get_count(years)
print(df_count)